# ABS Quarterly Consumer Price Index 6401

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import pandas as pd
import readabs as ra
from readabs import metacol as mc

In [2]:
# local imports
from abs_helper import (
    get_abs_data,
    ANNUAL_CPI_TARGET_RANGE,
    QUARTERLY_CPI_TARGET,
    QUARTERLY_CPI_RANGE,
)
from abs_summary_plot import plot_summary
from plotting import (
    finalise_plot,
    line_plot,
    plot_growth_finalise,
)
from utility import annualise_percentages
from henderson import hma

In [3]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [4]:
abs_dict, meta, source, RECENT = get_abs_data("6401.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

In [5]:
meta[mc.table].unique()

array(['640101', '640102', '640103', '640104', '640105', '640106',
       '640107', '640108', '640109', '640110', '640111', '640112'],
      dtype=object)

## Plot

In [6]:
TARGETS = {"axhspan": ANNUAL_CPI_TARGET_RANGE, "axhline": QUARTERLY_CPI_TARGET}

### Summary chart

In [7]:
def summarise() -> None:
    """Summarise the ABS data."""

    required = {
        # order from least to most important (ie. bottom to top)
        # chart label: (ABS series ID code, growth over n periods)
        "1Q growth: Tradable CPI": ("A2330526L", 1),
        "4Q growth: Tradable CPI": ("A2330526L", 4),
        "1Q growth: Non-tradable CPI": ("A2330571X", 1),
        "4Q growth: Non-tradable CPI": ("A2330571X", 4),
        "1Q growth: Goods component CPI": ("A2330616T", 1),
        "4Q growth: Goods component CPI": ("A2330616T", 4),
        "1Q growth: Services component CPI": ("A2330706W", 1),
        "4Q growth: Services component CPI": ("A2330706W", 4),
        "1Q growth: Headline CPI": ("A2325846C", 1),
        "4Q growth: Headline CPI": ("A2325846C", 4),
        "1Q growth: Headline CPI (SA)": ("A3604506F", 1),
        "4Q growth: Headline CPI (SA)": ("A3604506F", 4),
        "1Q growth: Weighted median CPI (SA)": ("A3604503X", 1),
        "4Q growth: Weighted median CPI (SA)": ("A3604503X", 4),
        "1Q growth: Trimmed mean CPI (SA)": ("A3604509L", 1),
        "4Q growth: Trimmed mean CPI (SA)": ("A3604509L", 4),
    }

    plot_summary(
        required,
        abs_dict,
        meta,
        start="2000Q1",
        title=f"Key inflation statistics for {abs_dict['640102'].index[-1]}",
        rfooter=source,
        lfooter="Australia. All values are percentages. "
        "Seasonally adjusted series are marked (SA). ",
        show=SHOW,
        verbose=False,
    )


summarise()

Overriding x0: True


### Headline / Trimmed Mean / Weighted Median - Annual+Quarterly

Also volatility

In [8]:
# Used for pattern matching retrieval
QUARTERLY_PHRASE = "Percentage Change from Previous Period"
ANNUAL_PHRASE = "Percentage Change from Corresponding Quarter of Previous Year"
INDEX_PHRASE = "Index Numbers"
AUS_PHRASE = "Australia"

In [9]:
def plot_volatility(
    store: dict[str, pd.Series], starts: dict[str, pd.Period], table: str
) -> None:
    """Plot the stability/volatility of the different quarterly CPI series."""

    # calculate the volatility/stability of the quarterly growth rates
    vol_from = pd.Series(starts).max() + 1
    frame = pd.DataFrame(store)
    frame = frame[frame.index >= vol_from]
    ddof = 1
    qv_series = frame.diff(periods=1).std(ddof=ddof)
    qc_series = pd.Series(
        {name: series.corr(series.shift(1)) for name, series in frame.items()}
    )

    # calculate and plot trends, calculate mean absolute errors, MAE and SD of AE
    hma_term = 9
    hma_frame = pd.DataFrame(
        [hma(frame[x], hma_term) for x in frame.columns], index=frame.columns
    ).T
    for col in hma_frame.columns:
        svt = pd.DataFrame(
            [frame[col], hma_frame[col]],
            index=[
                "Series - quarter on quarter growth rate",
                f"{hma_term}-term Henderson moving average",
            ],
        ).T
        title = col.replace("Seasonally Adjusted", "(SA)").replace("Original", "(Orig)")
        line_plot(
            svt,
            title=f"Series v Trend: {title}",
            ylabel="Per cent per quarter",
            width=[1, 2.5],
            color=["darkorange", "blue"],
            axhspan=QUARTERLY_CPI_RANGE,
            rfooter=f"{source} {table}",
            lfooter="Australia. Growth rates calculated from the raw index. ",
            y0=True,
            show=SHOW,
        )
    absolute_errors = (frame - hma_frame).abs()
    q_mae_series = absolute_errors.mean().sort_values()
    q_maesd_series = absolute_errors.std(ddof=1).sort_values()

    # utility plotting function
    def start_plotting(s: pd.Series) -> plt.Axes:
        color = "cornflowerblue"
        s = s.sort_values()
        names = {
            x: x.replace("Seasonally Adjusted", "(SA)").replace("Original", "(Orig)")
            for x in s.index
        }
        s = s.rename(index=names)
        ax = s.plot.barh(color=color)
        for y, x in enumerate(s.values):
            t = ax.text(
                0.01, y, f"{x:.2f}", color="white", va="center", fontsize="xx-small"
            )
            t.set_path_effects([pe.withStroke(linewidth=1, foreground=color)])
        ax.set_yticklabels(s.index, fontsize="x-small")
        return ax

    # plot degree of autocorrelation in the quarterly growth rates
    ax = start_plotting(qc_series)
    finalise_plot(
        ax,
        title="Stability of Quarterly CPI Measures",
        xlabel="Pearson correlation for the quarterly growth\n"
        "rate and that for the previous quarter",
        ylabel=None,
        rfooter=f"{source} {table}",
        lfooter="Australia. Growth rates calculated from the raw index. "
        f"Data from {vol_from} to {frame.index[-1]}. "
        "Higher is better. ",
        show=SHOW,
    )

    # plot the volatility of the quarterly growth rate series
    ax = start_plotting(qv_series)
    finalise_plot(
        ax,
        title="Volatility of Quarterly CPI Measures",
        xlabel="Standard deviation of first differences for\n"
        "quarterly CPI growth rates (percentage points)",
        ylabel=None,
        rfooter=f"{source} {table}",
        lfooter="Australia. Growth rates calculated from the raw index. "
        f"Std. Dev. DoF={ddof}. "
        f"Data from {vol_from} to {frame.index[-1]}. "
        "Lower is better. ",
        show=SHOW,
    )

    # plot mean absolute error between the series and the trend
    ax = start_plotting(q_mae_series)
    finalise_plot(
        ax,
        title="Mean Absolute Error\nfor Quarterly CPI Measures",
        xlabel="Mean absolute error between the series and\n"
        f"the {hma_term}-term Henderson moving average",
        ylabel=None,
        rfooter=f"{source} {table}",
        lfooter="Australia. Growth rates calculated from the raw index. "
        f"Std. Dev. DoF={ddof}. "
        f"Data from {vol_from} to {frame.index[-1]}. "
        "Lower is better. ",
        show=SHOW,
    )

    # plot standard deviation of the absolute errors between the series and the trend
    ax = start_plotting(q_maesd_series)
    finalise_plot(
        ax,
        title="Distribution of Absolute Errors\nfor Quarterly CPI Measures",
        xlabel="Std. Dev. of absolute errors between series\n"
        f"and the {hma_term}-term Henderson moving average",
        ylabel=None,
        rfooter=f"{source} {table}",
        lfooter="Australia. Growth rates calculated from the raw index. "
        f"Std. Dev. DoF={ddof}. "
        f"Data from {vol_from} to {frame.index[-1]}. "
        "Lower is better. ",
        show=SHOW,
    )

In [10]:
def analytic_charts():
    """Produce charts for the analytic series:
    - Annualised growth rates for selected CPI series
    - Annual and quarterly growth rates for selected CPI series
    - Annual and quarterly growth rates for selected CPI series calculated from the raw index
    - Volatility of the quarterly growth rates for selected CPI series."""

    # set-up
    phrases = (ANNUAL_PHRASE, QUARTERLY_PHRASE, INDEX_PHRASE)
    starts, store = {}, {}
    table = "640106"
    data_t8 = abs_dict[table]
    wanted_charts = [
        ("Trimmed Mean", "Seasonally Adjusted"),
        ("Weighted Median", "Seasonally Adjusted"),
        ("All groups CPI ;  Australia", "Original"),  # Headline CPI
        ("All groups CPI", "Seasonally Adjusted"),  # Headline CPI seasonally adjusted
        ("All groups CPI excluding 'volatile items'", "Original"),
        ("All groups CPI excluding food and energy", "Original"),
        ("All groups, goods component", "Original"),
        ("All groups, services component", "Original"),
        ("Tradables", "Original"),
        ("Non-tradables", "Original"),
    ]
    for index, (label, series_type) in enumerate(wanted_charts):
        print(f"Processing {label} {series_type} {table}")
        title = f"CPI: {label}"

        # select data series identifiers
        a_id, q_id, i_id = [
            ra.find_abs_id(
                meta,
                {
                    table: mc.table,
                    series_type: mc.stype,
                    phrase: mc.did,
                    label: mc.did,
                },
            )[
                1
            ]  # the second element is the identifier
            for phrase in phrases
        ]

        # plot the annualised growth rates using ABS publised data
        comparative = pd.DataFrame(
            (data_t8[a_id], annualise_percentages(data_t8[q_id], 4)),
            index=("Annual rate", "Quarterly annualised rate"),
        ).T
        line_plot(
            comparative,
            starts=plot_times[1:],
            title=f"{title} (annualised)",
            ylabel="Per cent per year",
            y0=True,
            tags="annual-v-annualised",
            pre_tag=f"main-{index}-",
            axhspan=ANNUAL_CPI_TARGET_RANGE,
            rfooter=f"{source} {table}",
            lfooter=f"Australia. {series_type.capitalize()} series. ",
            width=(1.5, 2),
            show=SHOW,
        )

        # plot annual and quarterly growth rates using ABS published data
        for plot_from, plot_tag in zip(plot_times, plot_tags):
            p_plot_from = (
                None
                if plot_from is None
                else pd.Period(plot_from, freq=data_t8.index.freq)
            )
            # plot growth rates published bu ABS
            plot_growth_finalise(
                data_t8[a_id],
                data_t8[q_id],
                from_=p_plot_from,
                title=title,
                **TARGETS,
                tag=f'-{series_type.replace(" ", "-")}{plot_tag}',
                rfooter=f"{source} {table}",
                lfooter=f"Australia. {series_type.capitalize()} series. ",
                show=SHOW,
                annotate="x-small",
            )

        # plot growth rates calculated from the ABS raw index (more precision)
        raw_index = data_t8[i_id]
        plot_growth_finalise(
            raw_index.pct_change(4) * 100,  # derived annual growth rates
            (derived_qtly := raw_index.pct_change(1) * 100),
            from_=RECENT,
            title=f"{title} [calculated]",
            **TARGETS,
            rfooter=f"{source} {table}",
            lfooter=f"Australia. {series_type.capitalize()} series. "
            + "Growth rates calculated from the raw index. ",
            annotate="x-small",
            annotation_rounding=2,
            show=SHOW,
        )

        # collect data for volatility/stability analysis
        name = f"{label} {series_type}"
        store[name] = derived_qtly
        starts[name] = raw_index.first_valid_index()

    plot_volatility(store, starts, table)


analytic_charts()

Processing Trimmed Mean Seasonally Adjusted 640106
Processing Weighted Median Seasonally Adjusted 640106
Processing All groups CPI ;  Australia Original 640106
Processing All groups CPI Seasonally Adjusted 640106
Processing All groups CPI excluding 'volatile items' Original 640106
Processing All groups CPI excluding food and energy Original 640106
Processing All groups, goods component Original 640106
Processing All groups, services component Original 640106
Processing Tradables Original 640106
Processing Non-tradables Original 640106


### Analytical annual series - comparisons

In [11]:
def analytic_annual():
    """Produce charts for analytic data."""

    table = "640106"
    data = abs_dict[table]
    stem = "Percentage Change from Corresponding Quarter of Previous Year ;  "
    post = " ;  Australia ;"
    groups = [
        ["Tradables", "Non-tradables"],
        ["Original", "Seasonally Adjusted"],
        ["Goods", "Services"],
        ["Trimmed Mean", "Weighted Median"],
        ["Original", "Trimmed Mean"],
    ]

    # manage long/ugly names
    replacements = {
        "Goods": "All groups, goods component",
        "Services": "All groups, services component",
        "Original": "All groups CPI",
        "Seasonally Adjusted": "All groups CPI, seasonally adjusted",
    }

    for dataset in groups:
        idents = []
        for title in dataset:
            title = replacements.get(title, title)
            description = f"{stem}{title}{post}"
            series_type = (
                "Seasonally Adjusted"
                if "season" in title or "Trimmed" in title or "Weighted" in title
                else "Original"
            )
            search = {
                table: mc.table,
                description: mc.did,
                series_type: mc.stype,
            }
            table, ident, _ = ra.find_abs_id(meta, search)
            idents.append(ident)

        for start, tag in zip(plot_times, plot_tags):
            df = (
                data[data.index >= pd.Period(start, freq=data.index.freq)]
                if start
                else data
            )
            pair = df[idents]
            pair = pair.dropna(how="all", axis=0)
            pair.columns = dataset
            pair = pair.dropna(how="any", axis=0)  # compare where we can
            ax = pair.plot(lw=2, color=["blue", "darkorange"])
            finalise_plot(
                ax,
                title=f'CPI: {", ".join(dataset)}',
                ylabel="YoY Growth (per cent)",
                tag=tag,
                axhspan=ANNUAL_CPI_TARGET_RANGE,
                legend={"loc": "best", "fontsize": "x-small"},
                rfooter=f"{source} {table}",
                lfooter="Australia. Per cent change from same quarter in previous year",
                show=SHOW,
            )


analytic_annual()

### Expenditure groups, sub-groups and classes

#### Map of groups, sub-groups, classses

In [12]:
expenditure_groups = {
    "Food and non-alcoholic beverages",
    "Alcohol and tobacco",
    "Clothing and footwear",
    "Housing",
    "Furnishings, household equipment and services",
    "Health",
    "Transport",
    "Communication",
    "Recreation and culture",
    "Education",
    "Insurance and financial services",
}

expenditure_sub_groups = {
    "Bread and cereal products",
    "Meat and seafoods",
    "Dairy and related products",
    "Fruit and vegetables",
    "Food products n.e.c.",
    "Non-alcoholic beverages",
    "Meals out and take away foods",
    "Alcoholic beverages",
    "Tobacco",
    "Garments",
    "Footwear",
    "Accessories and clothing services",
    "Rents",
    "New dwelling purchase by owner-occupiers",
    "Other housing",
    "Utilities",
    "Furniture and furnishings",
    "Household textiles",
    "Household appliances, utensils and tools",
    "Non-durable household products",
    "Domestic and household services",
    "Medical products, appliances and equipment",
    "Medical, dental and hospital services",
    "Private motoring",
    "Urban transport fares",
    "Communication",
    "Audio, visual and computing equipment and services",
    "Newspapers, books and stationery",
    "Holiday travel and accommodation",
    "Other recreation, sport and culture",
    "Education",
    "Insurance",
    "Financial services",
}

expenditure_classes = {
    "Bread",
    "Cakes and biscuits",
    "Breakfast cereals",
    "Other cereal products",
    "Beef and veal",
    "Pork",
    "Lamb and goat",
    "Poultry",
    "Other meats",
    "Fish and other seafood",
    "Milk",
    "Cheese",
    "Ice cream and other dairy products",
    "Fruit",
    "Vegetables",
    "Eggs",
    "Jams, honey and spreads",
    "Food additives and condiments",
    "Oils and fats",
    "Snacks and confectionery",
    "Other food products n.e.c.",
    "Coffee, tea and cocoa",
    "Waters, soft drinks and juices",
    "Restaurant meals",
    "Take away and fast foods",
    "Spirits",
    "Wine",
    "Beer",
    "Tobacco",
    "Garments for men",
    "Garments for women",
    "Garments for infants and children",
    "Footwear for men",
    "Footwear for women",
    "Footwear for infants and children",
    "Accessories",
    "Cleaning, repair and hire of clothing and footwear",
    "Rents",
    "New dwelling purchase by owner-occupiers",
    "Maintenance and repair of the dwelling",
    "Property rates and charges",
    "Water and sewerage",
    "Electricity",
    "Gas and other household fuels",
    "Furniture",
    "Carpets and other floor coverings",
    "Household textiles",
    "Major household appliances",
    "Small electric household appliances",
    "Glassware, tableware and household utensils",
    "Tools and equipment for house and garden",
    "Cleaning and maintenance products",
    "Personal care products",
    "Other non-durable household products",
    "Child care",
    "Hairdressing and personal grooming services",
    "Other household services",
    "Pharmaceutical products",
    "Therapeutic appliances and equipment",
    "Medical and hospital services",
    "Dental services",
    "Motor vehicles",
    "Spare parts and accessories for motor vehicles",
    "Automotive fuel",
    "Maintenance and repair of motor vehicles",
    "Other services in respect of motor vehicles",
    "Urban transport fares",
    "Postal services",
    "Telecommunication equipment and services",
    "Audio, visual and computing equipment",
    "Audio, visual and computing media and services",
    "Books",
    "Newspapers, magazines and stationery",
    "Domestic holiday travel and accommodation",
    "International holiday travel and accommodation",
    "Equipment for sports, camping and open-air recreation",
    "Games, toys and hobbies",
    "Pets and related products",
    "Veterinary and other services for pets",
    "Sports participation",
    "Other recreational, sporting and cultural services",
    "Preschool and primary education",
    "Secondary education",
    "Tertiary education",
    "Insurance",
    "Deposit and loan facilities (direct charges)",
    "Other financial services",
}

In [13]:
def category_checks():
    """sanity checks on above maps."""

    print(
        "Check count mapped items: "
        f"{len(expenditure_groups) + len(expenditure_sub_groups) + len(expenditure_classes)}"
    )

    # identify any missing
    table = "640105"
    search = {
        table: mc.table,
        AUS_PHRASE: mc.did,
        INDEX_PHRASE: mc.did,
    }
    rows = ra.search_abs_meta(meta, search)

    items = (
        rows[mc.did]
        .str.replace(";  Australia ;", "")
        .str.replace("Index Numbers ;", "")
        .str.strip()
    ).to_list()
    print(f"Check count ABS columns: {len(items)} - should be one more than above")

    print("Missing [should be only 'All groups CPI'] ...")
    for i in items:
        if (
            i not in expenditure_classes
            and i not in expenditure_sub_groups
            and i not in expenditure_groups
        ):
            print(i)

    clusters = (expenditure_groups, expenditure_sub_groups, expenditure_classes)
    for cluster in clusters:
        for element in cluster:
            if element not in items:
                print(cluster)


category_checks()

Check count mapped items: 131
Check count ABS columns: 132 - should be one more than above
Missing [should be only 'All groups CPI'] ...
All groups CPI


## plot trend lines

In [14]:
def get_cpi_index_elements(
    level: str = "class",
) -> pd.DataFrame:
    """Retrieve the CPI index elements at the specified level."""

    # set-up
    table = "640105"
    data = pd.DataFrame()

    # sanity checks
    levels = ("group", "sub-group", "class")
    if level not in levels:
        print(f"Warning: level should be one of {levels}")
        return data

    # compile DataFrame of required data items
    at_level = {
        "class": expenditure_classes,
        "sub-group": expenditure_sub_groups,
        "group": expenditure_groups,
    }[level]

    for cluster in at_level:
        id_selector = {
            table: mc.table,
            "Index Numbers": mc.did,
            cluster: mc.did,
        }

        _, series_id, _ = ra.find_abs_id(
            meta, search_terms=id_selector, validate_unique=False
        )
        data[cluster] = abs_dict[table][series_id]

    return data

In [15]:
def inflation_by_groups():
    """Plot CPI inflation by groups."""

    data = get_cpi_index_elements("group").sort_index(axis=1)
    if len*(data) == 0:
        return  # no data
    pre_covid = pd.Period("2019-12-01", freq="Q")
    ten_years_ago = data.index[-1] - 40

    for span in ten_years_ago, pre_covid:
        select = data.div(data.loc[span]).mul(100)
        select = select[select.index >= span]
        markers = " ", "o", "s", "d", "p", "*", "<", ">", "v", "^", "P", "X"
        line_plot(
            select,
            title=f"CPI Inflation by Groups (Index: {span}=100)",
            ylabel="Index",
            marker=markers,
            markersize=5,
            legend={"loc": "upper left", "fontsize": "xx-small", "ncols": 2},
            axhline={"y": 100, "color": "black", "linestyle": "--", "linewidth": 0.75},
            width=2,
            rfooter=source,
            lfooter="Australia. Original series.",
            show=SHOW,
        )


inflation_by_groups()

TypeError: unsupported operand type(s) for *: 'builtin_function_or_method' and 'float'

#### Plot latest annual and quarterly growth by expenditure group

In [ ]:
def get_cpi_growth_elements(
    level: str = "class", mode: str = "annual"
) -> pd.DataFrame | None:
    """Calculate expenditure growth by group, subgroup or class,
    for a quarter, through the year, or since COVID [Q4 2019]."""

    # sanity checks
    modes = ("annual", "quarterly", "post-covid")
    if mode not in modes:
        print(f"Warning: mode should be one of {modes}")
        return None

    data = get_cpi_index_elements(level)
    match mode:
        case "annual":
            periods = 4
        case "quarterly":
            periods = 1
        case "post-covid":
            start = pd.Period("2019Q4", freq="Q")
            periods = (data.index[-1] - start).n

    data = ((data / data.shift(periods) - 1) * 100)

    return data

In [ ]:
def group_growth():
    """Bar chart of latest growth in CPI groups and sub-groups."""

    table = "7"
    for level in "group", "sub-group":
        figsize = (9, 10) if level == "sub-group" else (9, 4.5)
        for mode in "annual", "quarterly", "post-covid":
            data = get_cpi_growth_elements(level=level, mode=mode)
            quarter = str(pd.Period(data.index[-1], freq="Q")).replace("Q", " Q")
            data = data.iloc[-1].sort_values()

            ax = data.plot.barh()
            title = f"{mode.capitalize()} Growth by\nCPI {level.capitalize()}"
            finalise_plot(
                ax,
                title=title,
                xlabel="Per cent",
                rfooter=f"{source} {table}",
                lfooter=f"Australia. {quarter}. Original series.",
                figsize=figsize,
                show=SHOW,
            )


group_growth()

#### Plot top/bottom growing expenditure classes

In [ ]:
def class_growth():
    """Bar chart of latest growth in expenditure classes."""

    top_keep = 30
    table = "7"

    for mode in "annual", "quarterly", "post-covid":
        data = get_cpi_growth_elements(level="class", mode=mode)
        quarter = str(pd.Period(data.index[-1], freq="Q")).replace("Q", " Q")

        for tag in ("top", "last"):
            subset = (
                data.iloc[-1].sort_values(ascending=True)[-top_keep:]
                if tag == "top"
                else data.iloc[-1].sort_values(ascending=True)[:top_keep]
            )

            ax = subset.plot.barh()
            title = f"{mode.capitalize()} Growth by CPI\nExpenditure Class - {tag} {top_keep}"
            finalise_plot(
                ax,
                title=title,
                tag=tag,
                xlabel="Per cent",
                rfooter=f"{source} {table}",
                lfooter=f"Australia. {quarter}. "
                f"Note: there are {len(expenditure_classes)} expenditure classes",
                figsize=(9, 8),
                show=SHOW,
            )


class_growth()

#### Breadth of inflation across CPI classes

In [ ]:
def breadth_growth():
    """Chart the breadth of growth in CPI classes."""

    # set-up
    table = "7"
    annual_thresh = 3.0  # per cent - upper policy target
    # compound quarterisation ...
    qrtly_thresh = ((((annual_thresh / 100) + 1) ** 0.25) - 1) * 100

    for modality in "annual", "quarterly":
        # get the data
        data = get_cpi_growth_elements(level="class", mode=modality)

        # calculate breadth
        thresh = annual_thresh if modality == "annual" else qrtly_thresh
        rounding = 1 if modality == "annual" else 3
        inflating_count = (data > thresh).sum(axis=1, skipna=True)
        total = data.notna().sum(axis=1)
        breadth = inflating_count / total * 100

        # plot results
        ax = breadth.plot(lw=2.5)
        finalise_plot(
            ax,
            title=f"CPI exp. classes with {modality} "
            f"price growth > {round(thresh, rounding)}%",
            ylabel="Per cent",
            rfooter=f"{source} {table}",
            lfooter="Australia. Original series. "
            "CPI expenditure classes only. "
            f"Endpoint: {breadth.iloc[-1]:0.2f}% of CPI expenditure classes. ",
            show=SHOW,
        )


breadth_growth()

### Tails in CPI expenditure classes

In [ ]:
def tails():
    """Chart tails in the CPI expenditure classes."""

    # set-up
    table = "640105"
    data = abs_dict[table]
    base_start = data.index[-1].year - 11

    for mode in "annual", "quarterly":
        data = get_cpi_growth_elements(level="class", mode=mode)
        base = data[(data.index.year >= base_start)]

        ax = base.T.quantile(q=[0.1, 0.5, 0.9], numeric_only=True).T.plot(lw=2.5)
        ax.legend(title="Quantiles", loc="upper left")
        finalise_plot(
            ax,
            title=f"{mode.capitalize()} Growth by CPI Expenditure Classes",
            tag="quartile",
            ylabel="Per cent",
            y0=True,
            rfooter=f"{source} {table}",
            lfooter="Australia. Original series. " "CPI expenditure classes only. ",
            show=SHOW,
        )


tails()

In [ ]:
def distro_plot(base: pd.DataFrame, bounds=None, **kwargs) -> None:
    """Produce distributional plots for the data.
    Assume **kwargs includes title, rfooter and lfooter."""

    bounds = 15 if bounds is None else bounds
    adj = base.median(axis=1).max()
    min_plot, max_plot = adj - bounds, bounds + adj
    min_data, max_data = base.min().min(), base.max().max()
    if min_data < min_plot or max_data > max_plot:
        min_plot, max_plot = max(min_plot, min_data), min(max_plot, max_data)
        kwargs["lfooter"] += "Larger outliers not plotted. "
    padding = 0.025
    adj = (max_plot - min_plot) * padding
    ylim = min_plot - adj, max_plot + adj

    ax = base.T.boxplot(rot=30)
    finalise_plot(
        ax,
        tag="box",
        ylabel="Per cent",
        ylim=ylim,
        show=SHOW,
        **kwargs,
    )

In [ ]:
def distro_recent(go_back_n_quarters: int = -20):
    """Chart recent N Quarters."""

    mode = "quarterly"
    data = get_cpi_growth_elements(level="class", mode=mode)
    if data is not None:
        base = data.iloc[go_back_n_quarters:]
        title = f"Distribution of {mode.capitalize()} Growth by CPI Exp. Class"
        rfooter = f"{source} 7"
        lfooter = "Australia. Original series. CPI expenditure classes only. "

        distro_plot(
            base,
            title=title,
            rfooter=rfooter,
            lfooter=lfooter,
            pre_tag="recent",
            axhline=QUARTERLY_CPI_TARGET,
            legend={"loc": "best", "fontsize": "x-small"},
        )

In [ ]:
distro_recent()

In [ ]:
def distro_q_on_q():
    """Chart comparisons with same quarter in previous years."""

    for mode in "annual", "quarterly":
        data = get_cpi_growth_elements(level="class", mode=mode)
        base_start = data.index[-1].year - 11
        current_qtr = data.index[-1].quarter
        base = data[
            (data.index.year >= base_start) & (data.index.quarter == current_qtr)
        ]
        title = f"Distribution of {mode.capitalize()} Growth by CPI Exp. Class"
        rfooter = f"{source} 7"
        lfooter = "Australia. Original series. CPI expenditure classes only. "
        bounds = 25 if mode == "annual" else 15

        distro_plot(
            base,
            bounds=bounds,
            title=title,
            rfooter=rfooter,
            lfooter=lfooter,
            axhline=QUARTERLY_CPI_TARGET,
            pre_tag="by-year",
            y0=True,
        )


distro_q_on_q()

## Exploratory

In [ ]:
def get_classes() -> pd.DataFrame:
    """Get Index values for each expenditure class."""

    table = "640105"
    search = {
        table: mc.table,
        AUS_PHRASE: mc.did,
        INDEX_PHRASE: mc.did,
    }
    rows = ra.search_abs_meta(meta, search)
    rows.index = pd.Index(
        rows[mc.did]
        .str.replace(";  Australia ;", "")
        .str.replace("Index Numbers ;", "")
        .str.strip()
    )
    rows = rows[~rows.index.duplicated(keep="last")]
    classes = {}
    for item in expenditure_classes:
        sid = rows.loc[item, mc.id]
        classes[item] = abs_dict[table][sid]

    return pd.DataFrame(classes)

## Finished

In [ ]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

In [ ]:
print("Finished")